In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Imports

In [3]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit
from scipy.signal import savgol_filter, butter

In [4]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [5]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [170]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [8]:
def load_Temp_init_bin_Lan(str_load, flag_print):      

    fid = open(str_load+'.bin',  'rb')   

    a    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    N_ions = aux[0]   

    b    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    j_save = aux[0]   

    c    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    dt_j_save_next = aux[0]   

    d    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    eta = aux[0]   

    e    = fromfile(fid,  int32,   1   )        # Read record start tag   
    Temp = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    f    = fromfile(fid,  int32,   1   )        # Read record start tag 
    save_T = fromfile(fid,  float64, 7*j_save)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag

    fid.close   
    
    print('len save_T', junk)   

    if flag_print == 1:
        print(a, b, c, d, e, f)   
        print('N_ions', N_ions)   
        print('j_save', j_save)   
        print('dt_j_save_next', dt_j_save_next)   
        print('eta', eta)   
        print('Temp', Temp)   


    save_T = reshape(save_T, (j_save, 7), order='F')   
    print(save_T[0])   

    return N_ions, j_save, dt_j_save_next, eta, Temp, save_T 

# Sélection fichier

In [32]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
Points deleted because they were not complete ['DC09_RF11']   2 pt(s)
Total number of data directories 114


In [171]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC01_RF08', 'DC01_RF09', 'DC01_RF10', 'DC01_RF11', 'DC03_RF08', 'DC03_RF09', 'DC03_RF10', 'DC03_RF11', 'DC05_RF08', 'DC05_RF09', 'DC05_RF10', 'DC05_RF11', 'DC07_RF08', 'DC07_RF09', 'DC07_RF10', 'DC07_RF11', 'DC09_RF08', 'DC09_RF09', 'DC09_RF10']
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC03_RF08
005 > DC03_RF09
006 > DC03_RF10
007 > DC03_RF11
008 > DC05_RF08
009 > DC05_RF09
010 > DC05_RF10
011 > DC05_RF11
012 > DC07_RF08
013 > DC07_RF09
014 > DC07_RF10
015 > DC07_RF11
016 > DC09_RF08
017 > DC09_RF09
018 > DC09_RF10
> condition names ['DC', 'RF']
> number of points 19
> N_ions = 1024
> e_GMol = 50


In [180]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power

data0 = data_name[0]
Tvar = PMandT[1]

> Points | 19
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try00/Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV
00 - 01  >  /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try01/Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV
00 - 02  >  /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try02/Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV
00 - 03  >  /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try03/Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV
00 - 04  >  /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try04/Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV
00 - 05  >  /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try05/Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV
Point n° 1
01 - 00  >  /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF09/Try00/Tem

Point n° 10
10 - 00  >  /home/adrian/RemoteFS/Hobitton/20200903/DC05_RF10/Try00/Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV
10 - 01  >  /home/adrian/RemoteFS/Hobitton/20200903/DC05_RF10/Try01/Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV
10 - 02  >  /home/adrian/RemoteFS/Hobitton/20200903/DC05_RF10/Try02/Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV
10 - 03  >  /home/adrian/RemoteFS/Hobitton/20200903/DC05_RF10/Try03/Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV
10 - 04  >  /home/adrian/RemoteFS/Hobitton/20200903/DC05_RF10/Try04/Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV
10 - 05  >  /home/adrian/RemoteFS/Hobitton/20200903/DC05_RF10/Try05/Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV
Point n° 11
11 - 00  >  /home/adrian/RemoteFS/Hobitton/20200903/DC05_RF11/Try00/Temp_SimuType2_01_N01024_Vrf0070_Udc0.3000D+01V_D1.0_S1.0RFG50eV
11 - 01  >  /home/adrian/RemoteFS/Hobitton/20

In [100]:
# Where SNR for each try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
19


In [101]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


# Plots
Nuage d'ions, Temp et Pos

## Load temperature from Langevin files

In [102]:
# eta = [1e-22, 2e-22, 4e-22, 8e-22, 1e-21, 2e-21, 4e-21, 8e-21, 1e-20, 2e-20, 4e-20]

# eta = [1e-20, 8e-21, 4e-21, 2e-21, 1e-21, 8e-22, 4e-22, 8e-19, 4e-19, 2e-19, 1e-19, 8e-20, 4e-20, 2e-20]
eta = 1.5e-20

In [103]:
## Data loading with hardcode names

plot_point = 2
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point][plot_try])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF10/Try00/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [104]:
N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T =\
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,1)

#print('N_ions',N_ions)
#print('j_save',j_save)
#print('dt*j_save_next',dt_j_save_next)
#print('eta',eta)
#print('Temp',Temp)
#print('save_T',save_T)
#print('shape(save_T)',shape(save_T))

#  Sacahnt que dans le Fortran
# dt   = 2d-9
# j_save_next  = 100
# eta = 1.5d-21
# Temperature = 0.5d-3

len save_T [55440]
[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]


In [50]:
# Get T and fluo PM  ALL SimuType
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],0,'coucou' )

# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

OSError: /home/adrian/RemoteFS/Hobitton/20200903/DC01_RF10/Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG50eV.dat not found.

In [126]:
# Get T and fluo PM  All SimuType
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
print(sort(onlyfiles))
only_init = [f for f in onlyfiles if "SimuTypeQ" in f]
ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,only_init[0].strip('.dat')[4:],flag_plot=0,fig_name='0')
# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

['Temp_SimuType2_01_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat']


In [132]:
len(ti[0]),len(ta[0])

(19860, 19860)

In [113]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)

address = all_subdir[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(data0[plot_point][0]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(data0[0][0]+'/'+'xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_quench = tmp_info[-2]
i_laser_fly = tmp_info[-1]

n_dt = 100
dt = 1/2e6

print(eta)
print(i_free__fly_Lan)
print(i_laser_fly_quench)
print(i_laser_fly)

/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF11/Try00
['Langevin_cooling.info'
 'xva_SimuType2_01_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuType4_01_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG.info']
1.5e-20
1000.0
8000.0
6000.0


In [181]:
n_dt = 100

i_free__fly_Lan = 1000 #2000
i_free__fly = 100
i_therm_fly = 2000

i_laser_fly = 6000 #2000
i_laser_fly_quench = 6000 #2000

i_cool__fly = 2000

dt = 1/2e6

In [144]:
# Plot temperature Langevin + md laser  ALL SimuType

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:] 
figure('Temperature'); clf()
ax1 = subplot(211)
ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')
,
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = subplot(212,sharex=ax1)
plot(tt_md*1e3,PM, label='Tx')
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time[ms]')
ylabel('Counts')

plt.tight_layout()
subplots_adjust(hspace=0.015)

In [145]:
# Plot temperature Langevin + md laser  ONLY INIT

tt_Lan = save_T[:,0] # linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta
T_aux = ti

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = to

#fig_name = file_name[-9:] 
figure('Temperature'); clf()
ax1 = subplot(211)
ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')
,
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = subplot(212,sharex=ax1)
ax2.plot(tt_md*1e3,PM, label='Tx')
# time_switch = i_free__fly_Lan*dt
# ax2.vlines(time_switch*1e3,1e-4,75,color='blue')
# time_switch = (i_free__fly_Lan+i_laser_fly)*dt
# ax2.vlines(time_switch*1e3,1e-4,75,color='orange')
# time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,75,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time[ms]')
ylabel('Counts')

plt.tight_layout()
subplots_adjust(hspace=0.015)

ValueError: all the input arrays must have same number of dimensions

In [156]:
plot_try

0

In [157]:
# Plot all temp for all conditions
fname = 'Temp_allpoints'


fig, ax = plt.subplots(5, 4,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,4): # b
    for k in range(0,5): # a
        plot_point = to_plot
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
        dirname_hc = str(data0[plot_point][plot_try])+'/'
        # Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
        datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
        N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
        load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
        # Get T and fluo PM
        onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
        ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,sort(onlyfiles)[2].strip('.dat')[4:],flag_plot = 0,fig_name = 'Temp_allpoints')

        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
        tt_md = ta[0]
        T_aux = te[0]
        
#         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
#         tt_md = ta[0]
#         T_aux = te[0]
#         tt = concatenate((tt_Lan,tt_md))
#         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        PM = to

        ax[k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
        ax[k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
        ax[k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#         ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
#         ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
# print(i_free__fly_Lan)
# print(i_laser_fly_quench)
# print(i_laser_fly)
        time_switch = i_free__fly_Lan*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='green')
        time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
        ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[k,l].set_yticks([1e-4,1e-2,1,1e2,1e4])
        ax[k,l].grid()
        #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 19:
            break
    if to_plot == 19:
        break

# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e4)
fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
savefig(fname+'.png') 

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/axes/_base.py:3449: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]
len save_T [55440]
[5.05000000e-07 1.91641179e-04 9.27131488e-05 3.40265653e-07
 5.61790996e+01 1.94840836e+00 7.47446665e+01]
len save_T [55440]
[5.05000000e-07 9.84295942e-05 1.57288810e-03 2.36631261e-06
 4.54875095e+01 1.26472602e+00 6.97114951e+01]
len save_T [55440]
[5.05000000e-07 3.20935101e-06 7.33268098e-05 1.18607059e-06
 4.36606825e+01 1.43932783e+00 6.75074007e+01]
len save_T [55440]
[5.05000000e-07 1.63388884e-06 2.58645559e-04 7.36150521e-06
 4.91874920e+01 1.86918232e+00 7.49301016e+01]
len save_T [55440]
[5.05000000e-07 3.96420799e-06 3.96351982e-04 5.30246372e-06
 4.36114051e+01 1.19837696e+00 

In [150]:
# Plot all temp one for each Eta
fname = 'Temp_allEta'

my_order = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # [6, 5, 4, 3, 2, 1, 0, 13, 12, 11, 10, 9, 8, 7]

fig, ax = plt.subplots(3, 4,num=fname,clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

plot_try = 0

plot_iter = 0
for l in range(0,4):
    for k in range(0,3):

        plot_point = my_order[plot_iter]
        # list(dict.fromkeys(points_and_coord))[plot_point]

#         str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
#         str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
#         str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        dirname_hc = str(data0[plot_point][plot_try])+'/'
        # Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
        datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
        N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
        load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
        # Get T and fluo PM
        onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
        ta, te, ti, to = plot_T_and_PM_Quen(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:])

        tt_Lan = save_T[:,0]
        tt_md = ta
        T_aux = ti

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        PM = to

        ax[k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
        ax[k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
        ax[k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
        ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
        ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
        time_switch = i_free__fly_Lan*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
        # ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[k,l].set_yticks([1e-4,1e-2,1,1e2,1e4])
        ax[k,l].grid()
        #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
        ax[k,l].annotate(list(dict.fromkeys(points_and_coord))[plot_point] + '\n' + str(eta[plot_point]),
                         xy=(0.73, 0.28), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc='w'))
        plot_iter += 1
        if plot_iter == 13:
            break
    if plot_iter == 13:
        break

# ax[3].set_xlim(-10,4000)
ax[k,l].set_ylim(2e-4,1e-1)
fig.suptitle('Temperature for many Eta parameters')
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
savefig(fname+'.png') 

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/axes/_base.py:3449: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


TypeError: 'float' object is not subscriptable

In [ ]:
# Plot all fluo rf relax après refroidissement

for patate,ketchup in enumerate(list(dict.fromkeys(points_and_coord))):

    plot_point = patate
    fname = 'Fluo_alltry_'+ketchup

    fig, ax = plt.subplots(3, 2,num=fname,clear=True,sharex=True,sharey=True)
    fig.set_size_inches(11.69,8.27)
    # suptitle('Saturation 866')


    to_plot = 0
    for l in range(0,2):
        for k in range(0,3):

            dirname_hc = str(data0[plot_point][to_plot])+'/'
            N_ions, j_save, dt_j_save_next, eta, Temp, save_T =\
            load_Temp_init_bin_Lan(dirname_hc+datainit_hc,0)
            # Get T and fluo PM
            onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
            ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],0,'coucou' )

            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run
            tt_md = ta[0]
            T_aux = te[0]

            tt = concatenate((tt_Lan,tt_md))
            T_all = concatenate((save_T[:,4:],T_aux),axis=0)

            PM = ti[0]

            ax[k,l].plot(tt_md*1e3,PM, label='Tx',color='C0')
            ax[k,l].vlines(ta[1]*1e3,1e-4,75,color='red')
            ax[k,l].vlines(ta[2]*1e3,1e-4,75,color='red')
            time_switch = i_free__fly_Lan*dt
            ax[k,l].vlines(time_switch*1e3,1e-4,75,color='blue')
            time_switch = (i_free__fly_Lan+i_laser_fly)*dt
            ax[k,l].vlines(time_switch*1e3,1e-4,75,color='orange')
            time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
            ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,75,color='purple')
            ax[k,l].grid()
        #     ax[k].set_ylim(-0.1,0.55)
            ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
            ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                    size=12, ha='left', va='top',
                    bbox=dict(boxstyle='round', fc='w'))
            to_plot += 1
            if to_plot == 6:
                break
        if to_plot == 6:
            break
    # ax[3].set_xlim(-10,4000),
    # ax[k,l].set_ylim(2e-4,1e-1),
    fig.suptitle(list(dict.fromkeys(points_and_coord))[plot_point])
    fig.text(0.56, 0.04, 'time [ms]', ha='center')
    fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

    # fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

    # fig.text(0.5, 0.01, 'f', ha='center')
    # fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

    fig.set_size_inches(11.69,8.27)
    plt.tight_layout()
    subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
    savefig(fname+'.png')

In [ ]:
for g,h in enumerate(points_and_coord):
    print(g,h)

In [168]:
ti

[array([36.77, 38.19, 38.46, ...,  5.12,  4.79,  4.91]),
 0.8558812995371294,
 5.176214147980156]

In [182]:
Gmol_data[2]

detect_rate = []
detect_threshold = 1

for k in range(shape(Gmol_data[2])[0]):
    print('k',k)
    detect_rate.append(0)
    for l,m in enumerate(Gmol_data[2][k]):
        print('... l',l)
        if m > detect_threshold:
            detect_rate[k] += 1
    detect_rate[k] = detect_rate[k] / shape(Gmol_data[2])[1]
    
for n,o in enumerate(list(dict.fromkeys(points_and_coord))):
    print(o)
    print(detect_rate[n])

k 0
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 1
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 2
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 3
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 4
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 5
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 6
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 7
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 8
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 9
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 10
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 11
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 12
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 13
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 14
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 15
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 16
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 17
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
k 18
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
DC0

In [162]:
detect_rate[0]

0.0

In [163]:
Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]

In [183]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(detect_rate[g])
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=25)

im = ax.scatter(x,y,
        marker='o',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap='rainbow')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(SNR_trig_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

# Distribution des vitesses

In [185]:
shape(all_subdir)

(114,)

In [192]:
dirname_hc

'/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try00/'

In [195]:
address = all_subdir[0]
dirname_hc = str(data0[0][0])+'/'
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[1].strip('.dat')[4:])
print(my_file)
r_LC,v_LC,a_LC = load_xyz_init_bin_DP(my_file)  

/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF08/Try00/xva_SimuType4_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV


In [205]:
shape(v_LC)

(3, 1024)

In [209]:
for i,j in enumerate(v_LC[0]):
    print(i,j)

0 88867.72514432964
1 -651.5660888610122
2 77004.06709718144
3 22160.906085478706
4 -36080.6034396929
5 30671.049725385
6 37491.63341398736
7 57282.841908587856
8 -30400.85091905025
9 19877.26242472162
10 108706.31117844701
11 -56447.88963546698
12 75046.73800320567
13 -62879.13471287289
14 53023.590708520336
15 -59648.71911776579
16 28506.37426015847
17 65870.88723219915
18 -85310.214623236
19 17579.509512792392
20 -58738.25894557117
21 87939.93183259272
22 -31997.467018624968
23 20983.667397305388
24 -87408.34281746787
25 -110421.40379622704
26 36018.790951519004
27 -90964.16459822436
28 -39588.71276222033
29 -21934.065017613815
30 -5135.329940466794
31 -107053.0702850932
32 23722.012946523642
33 77865.95299361684
34 79233.35170074776
35 -18441.956709708647
36 -14081.97833604346
37 108093.73437883945
38 -30047.047373866233
39 73364.94215900313
40 -56247.414883843434
41 16724.420503390986
42 25064.584325778458
43 3174.1772242767156
44 -67008.65217880257
45 -59951.31447424974
46 102310

859 66396.69628142052
860 88812.62795497636
861 -30183.121004272354
862 32532.752284808965
863 -26060.445492481816
864 -14245.654223983913
865 31758.875185553803
866 32113.161105545587
867 -81813.14251397114
868 32828.493640147484
869 -52045.83813669265
870 -48606.82705517468
871 -60168.336343721785
872 -54778.89010811913
873 25834.936431077014
874 -34367.67247518192
875 -62489.08497127003
876 -46777.67125660979
877 -15733.292415819527
878 102702.14992707345
879 -593.6890766022656
880 84832.02998494699
881 -27133.841799544796
882 606.0281753584366
883 -11267.47134035064
884 19133.378594281163
885 -90066.94049810483
886 -40356.31598554123
887 29667.034892023832
888 19937.360582882626
889 92923.99649317113
890 -59594.58382504519
891 40027.65123145497
892 -22016.58099761869
893 27802.842414288847
894 40982.11506582899
895 -44027.933857527765
896 -42193.72854215499
897 29668.337516208994
898 -149.0186156886299
899 -43688.645539253965
900 4865.018619032989
901 -3852.3067433211268
902 3844.9

In [254]:
# Recover Ca pos, speed and acc at the end of one specific step

dirname_hc = str(data0[4][2])
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(dirname_hc,sort(onlyfiles)[2].strip('.dat')[4:])
print(my_file)
r_LC,v_LC,a_LC = load_xyz_init_bin_DP(my_file)  

my_bins = arange(-2000,2000,100)

fig = figure('Terminal speed'); clf()
fig.suptitle(r'Atomic speed just before injection - '+str(dirname_hc[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
# my_bins = arange(-1500,1500,100)
ax1.hist(v_LC[0],bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=500))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
# my_bins = arange(-1500,1500,100)
ax2.hist(v_LC[1],bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=500))
ax2.grid()
xlabel('$v_y$ [m/s]')

ax3 = subplot(313)
# my_bins = arange(-1500,1500,100)
ax3.hist(v_LC[2],bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=500))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

/home/adrian/RemoteFS/Hobitton/20200903/DC03_RF08/Try02/xva_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
